In [1]:
#Step 1: load data.

In [2]:
#Step 1a: load data from ACS 
#using censusdata package and documentation 
#from here: https://github.com/jtleider/censusdata

In [3]:
import sys
sys.path.append('..')

In [4]:
from src.data.censusdata import censusdata

In [5]:
import pandas as pd

In [6]:
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)

In [7]:
#Search for table and variable combinations for desired stats.

In [8]:
acs_version = 'acs5' #or 'acs5'
acs_year = 2016

In [9]:
censusdata.search(acs_version, acs_year, 'label', '')

[('AIANHH', None, 'FIPS AIANHH code'),
 ('AIHHTLI', None, 'American Indian Trust Land/Hawaiian Home Land Indicator'),
 ('AITSCE', None, 'American Indian Tribal Subdivision (FIPS)'),
 ('ANRC', None, 'Alaska Native Regional Corporation (FIPS)'),
 ('B00001_001E',
  'UNWEIGHTED SAMPLE COUNT OF THE POPULATION',
  'Estimate!!Total'),
 ('B00002_001E', 'UNWEIGHTED SAMPLE HOUSING UNITS', 'Estimate!!Total'),
 ('B01001A_001E', 'SEX BY AGE (WHITE ALONE)', 'Estimate!!Total'),
 ('B01001A_002E', 'SEX BY AGE (WHITE ALONE)', 'Estimate!!Total!!Male'),
 ('B01001A_003E',
  'SEX BY AGE (WHITE ALONE)',
  'Estimate!!Total!!Male!!Under 5 years'),
 ('B01001A_004E',
  'SEX BY AGE (WHITE ALONE)',
  'Estimate!!Total!!Male!!5 to 9 years'),
 ('B01001A_005E',
  'SEX BY AGE (WHITE ALONE)',
  'Estimate!!Total!!Male!!10 to 14 years'),
 ('B01001A_006E',
  'SEX BY AGE (WHITE ALONE)',
  'Estimate!!Total!!Male!!15 to 17 years'),
 ('B01001A_007E',
  'SEX BY AGE (WHITE ALONE)',
  'Estimate!!Total!!Male!!18 and 19 years'),
 (

In [10]:
#search for race-related variables
#output supressed.

#censusdata.search(acs_version, acs_year, 'label', 'white')

In [11]:
#censusdata.printtable(censusdata.censustable(acs_version, acs_year, 'B02001'))

In [12]:
#censusdata.printtable(censusdata.censustable(acs_version, acs_year, 'B03002'))

In [13]:
#White, not Hispanic: B03002_003E (from total: B03002_001E)
#White, Hispanic: B03002_013E (from total: B03002_001E)
#Black: B02001_003E (from total: B02001_001E)
#Other: If totals are equal: (B02001_001E and B03002_001E) then other is total minus 
#White, not Hispanic plus White, Hispanic plus Black.

#Will check it out after finding geography codes.

In [14]:
#search for Socioeconomic status-related variables
#censusdata.search(acs_version, acs_year, 'label', 'degree')

In [15]:
#censusdata.printtable(censusdata.censustable(acs_version, acs_year, 'B15003'))

In [16]:
#censusdata.search(acs_version, acs_year, 'label', 'per capita income')

In [17]:
#censusdata.printtable(censusdata.censustable(acs_version, acs_year, 'B19301'))

In [18]:
#Finding geography codes for all counties in Texas

In [19]:
#output supressed.
#censusdata.geographies(censusdata.censusgeo([('state', '*')]), acs_version, acs_year)
#Texas is '48'
#censusdata.geographies(censusdata.censusgeo([('state', '48'), ('county', '*')]), acs_version, acs_year)
#Travis County is '453' (used for testing)

In [20]:
txcounties = censusdata.download(acs_version, acs_year,
                             censusdata.censusgeo([('state', '48'), ('county', '*')]),
                             ['B03002_001E','B02001_001E','B03002_003E','B03002_013E','B02001_003E', \
                              'B15003_022E','B15003_001E', \
                              'B19301_001E'])


In [21]:
#Check that total from the two tables are the same for all counties in Texas.
assert (txcounties.B03002_001E == txcounties.B02001_001E).all()

In [22]:
txcounties['population_'+str(acs_year)] = txcounties.B03002_001E
txcounties['fraction_white_nh'] = txcounties.B03002_003E / txcounties.B03002_001E
txcounties['fraction_hispanic'] = txcounties.B03002_013E / txcounties.B03002_001E
txcounties['fraction_black'] = txcounties.B02001_003E / txcounties.B02001_001E
txcounties['fraction_other_race'] = 1. - txcounties['fraction_white_nh'] - \
                                txcounties['fraction_hispanic'] - txcounties['fraction_black']

txcounties['fraction_w4yrdeg'] = txcounties.B15003_022E / txcounties.B15003_001E
txcounties['per_capita_income'] = txcounties.B19301_001E


In [23]:
txcounties = txcounties[['population_'+str(acs_year), 'fraction_white_nh', 'fraction_hispanic', \
                         'fraction_black','fraction_other_race', \
                         'fraction_w4yrdeg','per_capita_income']]
txcounties.describe()

,population_2016,fraction_white_nh,fraction_hispanic,fraction_black,fraction_other_race,fraction_w4yrdeg,per_capita_income
count,2.54e+02,2.54e+02,254.00,254.00,2.54e+02,254.00,254.00
mean,1.06e+05,5.72e-01,0.27,0.06,9.26e-02,0.13,24076.56
std,3.82e+05,2.11e-01,0.21,0.07,7.42e-02,0.05,4936.69
min,7.60e+01,7.60e-03,0.03,0.00,5.55e-17,0.03,12564.00
25%,7.05e+03,4.29e-01,0.14,0.01,5.04e-02,0.10,21083.00
50%,1.84e+04,6.03e-01,0.19,0.04,7.15e-02,0.12,23605.00
75%,4.94e+04,7.36e-01,0.36,0.09,1.13e-01,0.15,26674.75
max,4.43e+06,9.58e-01,0.94,0.34,5.32e-01,0.32,42236.00


In [24]:
#add column of county number (as state number and county number)

county_numbers = []
for value in txcounties.index.values:
    county_numbers.append(int('48'+value.geo[1][1]))

In [25]:
txcounties['county_code'] = county_numbers
txcounties = txcounties.set_index('county_code')

In [26]:
#load religiosity data.
import pandas as pd

In [27]:
religion = pd.read_excel('../data/external/U.S. Religion Census Religious '+\
                         'Congregations and Membership Study, 2010 (County File).XLSX')

#Citation:
#Association of Religious Data Archives (2010): Religious Congregations and Membership Study, 
#in: U.S. Religion Census (2010), County File, obtained on 2020/5/26
#from http://www.thearda.com/Archive/Files/Descriptions/RCMSCY10.asp

In [28]:
religion.keys()

Index(['TOTCNG', 'TOTADH', 'TOTRATE', 'EVANCNG', 'EVANADH', 'EVANRATE',
       'BPRTCNG', 'BPRTADH', 'BPRTRATE', 'MPRTCNG',
       ...
       'ZOROCNG', 'ZOROADH', 'ZORORATE', 'FIPS', 'STCODE', 'STABBR', 'STNAME',
       'CNTYCODE', 'CNTYNAME', 'POP2010'],
      dtype='object', length=568)

In [29]:
#Searching for Jewish keys
#filter_col = [col for col in religion if 'ADH' in col]
#filter_col

In [30]:
#pull columns for:
#Evangelical
#Catholic
#Mormon
#Jewish
#Total population in 2010
#County Code
religion = religion[religion['STABBR']=='TX'].loc[:, \
          ['TOTADH','EVANADH','BPRTADH',
             'MPRTADH','CATHADH','OTHADH','LDSADH','CJUDADH','OJUDADH','RJUDADH',\
           'POP2010','CNTYCODE','CNTYNAME']]

In [31]:
religion['county_code'] = religion['CNTYCODE']+48000

In [32]:
#Remove "County" from County column (this is a poor attempt b/c it removes the "Antonio" from "San Antonio")
#religion['County'] = religion['CNTYNAME'].str.split('\s').str[0]
#religion['County'] = religion['County'].str.upper()

In [33]:
religion['County'] = religion['CNTYNAME'].str.upper()

In [34]:
#convert number to fraction
religion['fraction_total_religion'] = religion['TOTADH'] / religion['POP2010']
religion['fraction_evangelical'] = religion['EVANADH'] / religion['POP2010']
religion['fraction_black_protestant'] = religion['BPRTADH'] / religion['POP2010']
religion['fraction_mainline'] = religion['MPRTADH'] / religion['POP2010']
religion['fraction_catholic'] = religion['CATHADH'] / religion['POP2010']
religion['fraction_other_religion'] = religion['OTHADH'] / religion['POP2010']
religion['fraction_mormon'] = religion['LDSADH'] / religion['POP2010']
religion['fraction_jewish'] = (religion['CJUDADH'] + religion['OJUDADH'] \
                               + religion['RJUDADH']) / religion['POP2010']
religion = religion.fillna(0)
religion['fraction_unclaimed_religion'] = 1. - religion['fraction_total_religion']

In [35]:
religion = religion[['county_code','fraction_evangelical', 'fraction_black_protestant', \
                     'fraction_catholic', 'fraction_mainline', \
                     'fraction_mormon','fraction_jewish','fraction_other_religion', \
                     'fraction_unclaimed_religion','County']]
religion.describe()

,county_code,fraction_evangelical,fraction_black_protestant,fraction_catholic,fraction_mainline,fraction_mormon,fraction_jewish,fraction_other_religion,fraction_unclaimed_religion
count,254.00,254.00,254.00,254.00,254.00,2.54e+02,2.54e+02,2.54e+02,254.00
mean,48254.00,0.35,0.01,0.17,0.08,6.86e-03,2.31e-05,1.21e-02,0.38
std,146.94,0.17,0.02,0.15,0.05,7.79e-03,2.17e-04,1.75e-02,0.17
min,48001.00,0.00,0.00,0.00,0.00,0.00e+00,0.00e+00,0.00e+00,-0.10
25%,48127.50,0.22,0.00,0.06,0.05,0.00e+00,0.00e+00,7.97e-05,0.28
50%,48254.00,0.34,0.00,0.12,0.07,5.23e-03,0.00e+00,7.36e-03,0.40
75%,48380.50,0.46,0.02,0.23,0.10,1.28e-02,0.00e+00,1.72e-02,0.50
max,48507.00,0.90,0.09,0.91,0.29,3.83e-02,2.56e-03,1.30e-01,0.93


In [36]:
#Find which counties have a negative percent other
import numpy as np
religion[religion['fraction_unclaimed_religion'] < 0].index

Int64Index([2571, 2591, 2605, 2631, 2670, 2749], dtype='int64')

In [37]:
#Set percent unclaimed to 0 for one county with a negative percent unclaimed
religion.at[[2571, 2591, 2605, 2631, 2670, 2749],'fraction_unclaimed_religion'] = 0

In [38]:
religion.describe()

,county_code,fraction_evangelical,fraction_black_protestant,fraction_catholic,fraction_mainline,fraction_mormon,fraction_jewish,fraction_other_religion,fraction_unclaimed_religion
count,254.00,254.00,254.00,254.00,254.00,2.54e+02,2.54e+02,2.54e+02,254.00
mean,48254.00,0.35,0.01,0.17,0.08,6.86e-03,2.31e-05,1.21e-02,0.38
std,146.94,0.17,0.02,0.15,0.05,7.79e-03,2.17e-04,1.75e-02,0.17
min,48001.00,0.00,0.00,0.00,0.00,0.00e+00,0.00e+00,0.00e+00,0.00
25%,48127.50,0.22,0.00,0.06,0.05,0.00e+00,0.00e+00,7.97e-05,0.28
50%,48254.00,0.34,0.00,0.12,0.07,5.23e-03,0.00e+00,7.36e-03,0.40
75%,48380.50,0.46,0.02,0.23,0.10,1.28e-02,0.00e+00,1.72e-02,0.50
max,48507.00,0.90,0.09,0.91,0.29,3.83e-02,2.56e-03,1.30e-01,0.93


In [39]:
religion = religion.set_index('county_code')

In [40]:
religion

,fraction_evangelical,fraction_black_protestant,fraction_catholic,fraction_mainline,fraction_mormon,fraction_jewish,fraction_other_religion,fraction_unclaimed_religion,County
county_code,,,,,,,,,
48001,0.35,0.02,0.08,4.24e-02,6.21e-03,0.0,6.24e-03,0.50,ANDERSON COUNTY
48003,0.38,0.00,0.14,6.54e-02,8.18e-03,0.0,8.18e-03,0.41,ANDREWS COUNTY
48005,0.41,0.02,0.12,6.90e-02,7.16e-03,0.0,3.71e-02,0.34,ANGELINA COUNTY
48007,0.19,0.00,0.18,7.03e-02,1.20e-02,0.0,1.20e-02,0.55,ARANSAS COUNTY
48009,0.48,0.00,0.11,5.40e-02,0.00e+00,0.0,1.10e-04,0.35,ARCHER COUNTY
...,...,...,...,...,...,...,...,...,...
48499,0.41,0.02,0.06,7.14e-02,1.06e-02,0.0,1.06e-02,0.42,WOOD COUNTY
48501,0.43,0.00,0.31,7.25e-02,0.00e+00,0.0,1.27e-04,0.18,YOAKUM COUNTY
48503,0.63,0.00,0.05,1.31e-01,1.56e-02,0.0,1.97e-02,0.17,YOUNG COUNTY


In [41]:
#let's get the land area of each county
land_area = pd.read_csv('https://query.data.world/s/bblueltx7fslj2s4gu65nyv5bcohuq') 
#a copy is saved in ../data/external/dataworld

In [42]:
#Load results from 2016 presidential election to get partisan lean
partisan_lean_2016 = pd.read_csv('https://query.data.world/s/5pczct6v6ltnjjz5m7lodlcetar5mh') 
#a copy is saved in ../data/external/dataworld
#partisan_lean_2016.info()

In [43]:
#load results from 2012 presidential election:
partisan_lean_2012 = pd.read_csv('https://query.data.world/s/7crbkiyggrinkda5ike6m4x6vzkk73') 
#a copy is saved in ../data/external/dataworld
partisan_lean_2012.County = partisan_lean_2012.County.astype(int)
#partisan_lean_2012.info()

In [44]:
#load results from 2008 presidential election:
partisan_lean_2008 = pd.read_csv('https://query.data.world/s/nrcqbzxe5blwoks5kfzo4xk2kqnn6k') 
#a copy is saved in ../data/external/dataworld
#partisan_lean_2008.info()

In [45]:
#remove AK county codes taht are strings
partisan_lean_2008 = partisan_lean_2008.loc[~(partisan_lean_2008['County'].astype(str).str[0] == 'A')]
partisan_lean_2008.County=partisan_lean_2008.County.astype(int)
#partisan_lean_2008.info()

In [46]:
partisan_lean_2016['margin_of_victory_D2016'] = partisan_lean_2016['dPct'] - partisan_lean_2016['rPct']
partisan_lean_2012['margin_of_victory_D2012'] = partisan_lean_2012['dPct'] - partisan_lean_2012['rPct']
partisan_lean_2008['margin_of_victory_D2008'] = partisan_lean_2008['dPct'] - partisan_lean_2008['rPct']

In [47]:
partisan_lean_2016 = partisan_lean_2016[['County','margin_of_victory_D2016']]
partisan_lean_2012 = partisan_lean_2012[['County','margin_of_victory_D2012']]
partisan_lean_2008 = partisan_lean_2008[['County','margin_of_victory_D2008']]

In [48]:
#let's merge the dfs
land_area = pd.merge(land_area, partisan_lean_2008, on='County')
land_area = pd.merge(land_area, partisan_lean_2012, on='County')
land_area = pd.merge(land_area, partisan_lean_2016, on='County')

In [49]:
land_area['county_code'] = land_area['County']

In [50]:
#land_area.loc[:,['County','TotalPopulation','LandAreaSqMiles'] \
#             ][land_area['County']==48261]

In [51]:
#get average of the margin of victory from 2012 and 2016
col = land_area.loc[: , ['margin_of_victory_D2008','margin_of_victory_D2012','margin_of_victory_D2016']]
land_area['margin_of_victory_D_avg'] = col.mean(axis=1)

In [52]:
land_area = land_area[['county_code','LandAreaSqMiles','margin_of_victory_D2008',\
                       'margin_of_victory_D2012','margin_of_victory_D2016','margin_of_victory_D_avg']]
land_area = land_area[(land_area['county_code'] > 48000) & (land_area['county_code'] < 49000)]

In [53]:
land_area = land_area.set_index('county_code')

In [54]:
land_area

,LandAreaSqMiles,margin_of_victory_D2008,margin_of_victory_D2012,margin_of_victory_D2016,margin_of_victory_D_avg
county_code,,,,,
48001,1062.60,-0.44,-0.52,-0.58,-0.51
48003,1500.72,-0.65,-0.63,-0.63,-0.64
48005,797.78,-0.35,-0.44,-0.47,-0.42
48007,252.08,-0.38,-0.43,-0.50,-0.44
48009,903.11,-0.65,-0.74,-0.79,-0.73
...,...,...,...,...,...
48499,645.24,-0.54,-0.64,-0.70,-0.63
48501,799.72,-0.63,-0.61,-0.60,-0.61
48503,914.47,-0.64,-0.72,-0.74,-0.70


In [55]:
all_features = pd.merge(txcounties, religion, on='county_code')
all_features = pd.merge(all_features, land_area, on='county_code')

In [56]:
all_features['population_density'] = all_features['population_'+str(acs_year)] \
/ all_features['LandAreaSqMiles']

In [57]:
all_features

,population_2016,fraction_white_nh,fraction_hispanic,fraction_black,fraction_other_race,fraction_w4yrdeg,per_capita_income,fraction_evangelical,fraction_black_protestant,fraction_catholic,...,fraction_jewish,fraction_other_religion,fraction_unclaimed_religion,County,LandAreaSqMiles,margin_of_victory_D2008,margin_of_victory_D2012,margin_of_victory_D2016,margin_of_victory_D_avg,population_density
county_code,,,,,,,,,,,,,,,,,,,,,
48001,57772,0.60,0.14,2.13e-01,0.05,0.08,18087,0.35,0.02,0.08,...,0.0,6.24e-03,0.50,ANDERSON COUNTY,1062.60,-0.44,-0.52,-0.58,-0.51,54.37
48003,17215,0.42,0.50,2.04e-02,0.05,0.07,29240,0.38,0.00,0.14,...,0.0,8.18e-03,0.41,ANDREWS COUNTY,1500.72,-0.65,-0.63,-0.63,-0.64,11.47
48005,87657,0.62,0.16,1.47e-01,0.08,0.10,21486,0.41,0.02,0.12,...,0.0,3.71e-02,0.34,ANGELINA COUNTY,797.78,-0.35,-0.44,-0.47,-0.42,109.88
48007,24729,0.68,0.20,1.67e-02,0.10,0.13,29426,0.19,0.00,0.18,...,0.0,1.20e-02,0.55,ARANSAS COUNTY,252.08,-0.38,-0.43,-0.50,-0.44,98.10
48009,8750,0.89,0.06,1.19e-02,0.04,0.17,29086,0.48,0.00,0.11,...,0.0,1.10e-04,0.35,ARCHER COUNTY,903.11,-0.65,-0.74,-0.79,-0.73,9.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48499,43198,0.83,0.06,5.23e-02,0.05,0.12,25353,0.41,0.02,0.06,...,0.0,1.06e-02,0.42,WOOD COUNTY,645.24,-0.54,-0.64,-0.70,-0.63,66.95
48501,8316,0.34,0.60,6.37e-03,0.05,0.13,22871,0.43,0.00,0.31,...,0.0,1.27e-04,0.18,YOAKUM COUNTY,799.72,-0.63,-0.61,-0.60,-0.61,10.40
48503,18275,0.79,0.15,1.26e-02,0.04,0.13,25837,0.63,0.00,0.05,...,0.0,1.97e-02,0.17,YOUNG COUNTY,914.47,-0.64,-0.72,-0.74,-0.70,19.98


In [58]:
#read in and load 2016 and 2020 Texas Democratic Primary results
results2020 = pd.read_csv('../data/external/Texas Primary Democratic Party Results 2020 - County.csv', \
           delimiter='\t', thousands=',')

In [59]:
#fix County typo found
results2020[results2020['County'] == 'LASALLE'].index

Int64Index([142], dtype='int64')

In [60]:
results2020.at[142,'County'] = 'LA SALLE'

In [61]:
results2020 = results2020[results2020.index > 0]

In [62]:
results2020['fraction_sanders_2020'] = results2020['SANDERS'].astype(float)/ \
 results2020['Total_Votes'].astype(float)
results2020['votes_sanders_2020'] = results2020['SANDERS']
results2020['votes_total_2020'] = results2020['Total_Votes']

In [63]:
results2020 = results2020[['County','fraction_sanders_2020','votes_sanders_2020','votes_total_2020']]

In [64]:
results2020

,County,fraction_sanders_2020,votes_sanders_2020,votes_total_2020
1,ANDERSON,0.21,336,1591
2,ANDREWS,0.36,84,231
3,ANGELINA,0.20,648,3249
4,ARANSAS,0.19,222,1179
5,ARCHER,0.17,28,167
...,...,...,...,...
250,WOOD,0.17,256,1470
251,YOAKUM,0.34,30,88
252,YOUNG,0.19,82,427
253,ZAPATA,0.26,814,3100


In [65]:
results2016 = pd.read_csv('../data/external/Texas Primary 2016 Results by County.csv', \
                          thousands=',')

In [66]:
#fix County typo found
results2016[results2016['County'] == 'Lasalle'].index

Int64Index([141], dtype='int64')

In [67]:
results2016.at[141,'County'] = 'La Salle'

In [68]:
results2016['Total_Votes'] = results2016['Clinton_Hillary']
other_dem_names = ['De La Fuente_Rocky', 'Hawes_Calvis', 'Judd_Keith', 'Locke_Star', \
                   "O'Malley_Martin", "Sanders_Bernie", "Wilson_Willie"]
for dem_name in other_dem_names:
    results2016['Total_Votes'] = results2016['Total_Votes'] + results2016[dem_name]




In [69]:
results2016['fraction_sanders_2016'] = results2016['Sanders_Bernie'].astype(float)/ \
 results2016['Total_Votes'].astype(float)
results2016['votes_sanders_2016'] = results2016['Sanders_Bernie']
results2016['votes_total_2016'] = results2016['Total_Votes']

In [70]:
results2016 = results2016[['County','fraction_sanders_2016','votes_sanders_2016','votes_total_2016']]

In [71]:
results2016.iloc[130:145]

,County,fraction_sanders_2016,votes_sanders_2016,votes_total_2016
130,Kenedy,0.30,32,106
131,Kent,0.47,28,60
132,Kerr,0.41,709,1748
133,Kimble,0.52,34,65
134,King,0.33,1,3
135,Kinney,0.31,107,347
136,Kleberg,0.26,630,2460
137,Knox,0.18,17,95
138,Lamar,0.29,388,1333
139,Lamb,0.27,87,323


In [72]:
#combine results dataframes on County.
#Capitalize County in results2016.
results2016['County'] = results2016['County'].str.upper()

In [73]:
results = pd.merge(results2016, results2020, on='County')

In [74]:
results['County'] = results['County'] + ' COUNTY'

In [75]:
#some debugging

In [76]:
#results

In [77]:
#results.isnull().sum()

In [78]:
#all_features

In [79]:
#all_features.isnull().sum()

In [80]:
#county1 = all_features.sort_values('County')['County']
#county2 = results.sort_values('County')['County']
#county2.index = county1.index

In [81]:
#county1[county1 != county2]

In [82]:
#county2[county1 != county2]

In [83]:
all_features_out = pd.merge(all_features, results, on='County')

In [84]:
all_features_out

,population_2016,fraction_white_nh,fraction_hispanic,fraction_black,fraction_other_race,fraction_w4yrdeg,per_capita_income,fraction_evangelical,fraction_black_protestant,fraction_catholic,...,margin_of_victory_D2012,margin_of_victory_D2016,margin_of_victory_D_avg,population_density,fraction_sanders_2016,votes_sanders_2016,votes_total_2016,fraction_sanders_2020,votes_sanders_2020,votes_total_2020
0,57772,0.60,0.14,2.13e-01,0.05,0.08,18087,0.35,0.02,0.08,...,-0.52,-0.58,-0.51,54.37,0.23,299,1317,0.21,336,1591
1,17215,0.42,0.50,2.04e-02,0.05,0.07,29240,0.38,0.00,0.14,...,-0.63,-0.63,-0.64,11.47,0.33,38,115,0.36,84,231
2,87657,0.62,0.16,1.47e-01,0.08,0.10,21486,0.41,0.02,0.12,...,-0.44,-0.47,-0.42,109.88,0.24,622,2646,0.20,648,3249
3,24729,0.68,0.20,1.67e-02,0.10,0.13,29426,0.19,0.00,0.18,...,-0.43,-0.50,-0.44,98.10,0.35,272,774,0.19,222,1179
4,8750,0.89,0.06,1.19e-02,0.04,0.17,29086,0.48,0.00,0.11,...,-0.74,-0.79,-0.73,9.69,0.38,64,168,0.17,28,167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,43198,0.83,0.06,5.23e-02,0.05,0.12,25353,0.41,0.02,0.06,...,-0.64,-0.70,-0.63,66.95,0.30,285,965,0.17,256,1470
250,8316,0.34,0.60,6.37e-03,0.05,0.13,22871,0.43,0.00,0.31,...,-0.61,-0.60,-0.61,10.40,0.48,10,21,0.34,30,88
251,18275,0.79,0.15,1.26e-02,0.04,0.13,25837,0.63,0.00,0.05,...,-0.72,-0.74,-0.70,19.98,0.39,132,342,0.19,82,427
252,14335,0.06,0.90,9.77e-03,0.03,0.05,16007,0.05,0.00,0.25,...,0.43,0.33,0.37,14.36,0.24,685,2906,0.26,814,3100


In [85]:
all_features_out.to_csv('../data/interim/all_features-'+acs_version+'-'+str(acs_year)+'.csv', index=False)